In [1]:
import ROOT
%jsroot on

Welcome to JupyROOT 6.08/02


In [2]:
ROOT.RooMsgService.instance().setGlobalKillBelow(5)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [3]:
import csv
import numpy as np
from matplotlib import pyplot
%matplotlib inline

# Getting Data. Defining a Model.

In [4]:
inputfile = 'input/training.csv'
all = list(csv.reader(open(inputfile,"rb"), delimiter=','))
sig = [float(row[1]) for row in all[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*50 for row in all[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in all[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in all[1:] if row[-1] == 'b' ]

In [5]:
sigsum = sum(sigweights)
bkgsum = sum(bkgweights)
print 'n signal {0}, n background {1}'.format(sigsum,bkgsum)

n signal 34599.4303856, n background 410999.847322


In [6]:
h1 = ROOT.TH1D("h1","signal histogram",40,0,200)
for i in range(len(sig)): h1.Fill(sig[i],sigweights[i])
h2 = ROOT.TH1D("h2","background histogram",40,0,200)
for i in range(len(bkg)): h2.Fill(bkg[i],bkgweights[i])
x = ROOT.RooRealVar("x","x",0,200)
l = ROOT.RooArgList(x)
signalhist = ROOT.RooDataHist("sighist", "sighist", l, h1)
sigpdf = ROOT.RooHistPdf("sigpdf","sigpdf",ROOT.RooArgSet(x),signalhist,0)
bkghist = ROOT.RooDataHist("bkghist", "bkghist", l, h2)
bkgpdf = ROOT.RooHistPdf("bkgpdf","bkgpdf",ROOT.RooArgSet(x),bkghist,0)

In [7]:
w = ROOT.RooWorkspace("w")
getattr(w,'import')(sigpdf)
getattr(w,'import')(bkgpdf)
w.var("x")
w.factory("SUM:model(n_sig[0.1*n_sig,10*n_sig]*sigpdf, n_bkg[.1*n_bkg,10*n_bkg]*bkgpdf)")
w.var("n_sig").setRange(0.1*sigsum,10*sigsum)
w.var("n_sig").setVal(sigsum)
w.var("n_bkg").setRange(.1*bkgsum,10*bkgsum)
w.var("n_bkg").setVal(bkgsum)
w.factory("ExtendPdf:background(bkgpdf,n_bkg)")
w.factory("ExtendPdf:signal(sigpdf,n_sig)")

<ROOT.RooExtendPdf object ("signal") at 0xb705490>

## Fitting Breit-Wigner

In [8]:
pdf = w.pdf("background")
x = w.var("x")

In [9]:
x.setBins(40)
data = pdf.generate(ROOT.RooArgSet(x))
data.SetName("data")
getattr(w,'import')(data)
data.Print()

RooDataSet::data[x] = 411000 entries


In [10]:
mean = ROOT.RooRealVar("mean","Mean of Gaussian",0,200)
sigma = ROOT.RooRealVar("sigma","Width of Gaussian",40,0,200)
ztautau = ROOT.RooBreitWigner("ztautau","Breit-Wigner(x,mean,sigma)",x,mean,sigma)
coef0 = ROOT.RooRealVar("c0","coefficient #0",1.0,-1.,1)
coef1 = ROOT.RooRealVar("c1","coefficient #1",0.1,-1.,1)
coef2 = ROOT.RooRealVar("c2","coefficient #2",-0.1,-1.,1)
fake = ROOT.RooChebychev("fake","background p.d.f.",x,ROOT.RooArgList(coef0,coef1,coef2))
fbkg = ROOT.RooRealVar("fbkg","background fraction",0.9,0.,1.)
model = ROOT.RooAddPdf('prod','prod',ROOT.RooArgList(ztautau,fake),ROOT.RooArgList(fbkg))
mean.setVal(91.1876)
model.fitTo(data,ROOT.RooFit.Range(0,200),ROOT.RooFit.PrintLevel(-1))

<ROOT.RooFitResult object at 0x(nil)>

In [11]:
mean.getVal()

95.79087760963627

In [12]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("Reconstructed Mass"))
plot.SetTitle("")
plot.SetXTitle("MMC mass (GeV)")
plot.GetXaxis().SetTitleSize(0.05)
plot.GetYaxis().SetTitleSize(0.05)
data.plotOn(plot,ROOT.RooFit.Name("data"))
model.plotOn(plot,ROOT.RooFit.Name("ztautau"))
bkg_component = ROOT.RooArgSet(fake)
model.plotOn(plot,ROOT.RooFit.Components(bkg_component),ROOT.RooFit.LineStyle(2),ROOT.RooFit.Name("fake"))

l = ROOT.TLegend( 0.56, 0.65, 0.9, 0.9)

dataobj = plot.findObject("data")
zobj = plot.findObject("ztautau")
fakeobj = plot.findObject("fake")
l.AddEntry( dataobj , "Data", "pl" )
l.AddEntry( fakeobj , "MultiJet", "l"  )
l.AddEntry(zobj, "Z#rightarrow#tau#tau #mu={0:.2f}".format(mean.getVal()),"l")
l.SetTextSize(0.03)
plot.Draw()
plot.SetTitle("")
l.Draw()
c.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).


In [13]:
c.SaveAs("output/BreitWigner.eps")

Info in <TCanvas::Print>: eps file output/BreitWigner.eps has been created


In [14]:
mean = ROOT.RooRealVar("mean","Mean of Gaussian",0,200)
sigma = ROOT.RooRealVar("sigma","Width of Gaussian",40,0,200)
ztautau = ROOT.RooBreitWigner("ztautau","Breit-Wigner(x,mean,sigma)",x,mean,sigma)
coef0 = ROOT.RooRealVar("c0","coefficient #0",1.0,-1.,1)
coef1 = ROOT.RooRealVar("c1","coefficient #1",0.1,-1.,1)
coef2 = ROOT.RooRealVar("c2","coefficient #2",-0.1,-1.,1)
fake = ROOT.RooChebychev("fake","background p.d.f.",x,ROOT.RooArgList(coef0,coef1,coef2))
fbkg = ROOT.RooRealVar("fbkg","background fraction",0.9,0.,1.)
model = ROOT.RooAddPdf('prod','prod',ROOT.RooArgList(ztautau,fake),ROOT.RooArgList(fbkg))
mean.setVal(91.1876)
mean.setConstant(1)
model.fitTo(data,ROOT.RooFit.Range(0,200),ROOT.RooFit.PrintLevel(-1))

<ROOT.RooFitResult object at 0x(nil)>

In [15]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("Reconstructed Mass"))
plot.SetTitle("")
plot.SetXTitle("MMC mass (GeV)")
plot.GetXaxis().SetTitleSize(0.05)
plot.GetYaxis().SetTitleSize(0.05)
data.plotOn(plot,ROOT.RooFit.Name("data"))
model.plotOn(plot,ROOT.RooFit.Name("ztautau"))
bkg_component = ROOT.RooArgSet(fake)
model.plotOn(plot,ROOT.RooFit.Components(bkg_component),ROOT.RooFit.LineStyle(2),ROOT.RooFit.Name("fake"))

l = ROOT.TLegend( 0.56, 0.65, 0.9, 0.9)

dataobj = plot.findObject("data")
zobj = plot.findObject("ztautau")
fakeobj = plot.findObject("fake")
l.AddEntry( dataobj , "Data", "pl" )
l.AddEntry( fakeobj , "MultiJet", "l"  )
l.AddEntry(zobj, "Z#rightarrow#tau#tau fixed #mu={0:.2f}".format(mean.getVal()),"l")
l.SetTextSize(0.03)
plot.Draw()
plot.SetTitle("")
l.Draw()
c.Draw()

In [16]:
c.SaveAs("output/MultiBackgroundPreselection.eps")

Info in <TCanvas::Print>: eps file output/MultiBackgroundPreselection.eps has been created


## Likelihood Fit - Stat only, Preselection

In [125]:
model = w.pdf("model")
data = model.generate(ROOT.RooArgSet(x))
model.fitTo(data,ROOT.RooFit.Range(0,200),ROOT.RooFit.PrintLevel(-1))

<ROOT.RooFitResult object at 0x(nil)>

In [126]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("Reconstructed Mass"))
plot.SetTitle("")
plot.SetXTitle("MMC mass (GeV)")
plot.GetXaxis().SetTitleSize(0.05)
plot.GetYaxis().SetTitleSize(0.05)

data.plotOn(plot)
model.plotOn(plot)
model.plotOn(plot,ROOT.RooFit.Components("bkgpdf"),ROOT.RooFit.Name("background"), ROOT.RooFit.LineStyle(2) )
model.plotOn(plot, ROOT.RooFit.Components("sigpdf"),ROOT.RooFit.Name("signal"), ROOT.RooFit.LineColor(2), ROOT.RooFit.LineStyle(2) )

#model.paramOn(plot,ROOT.RooFit.Layout(0.5,0.9,0.85))
vars = model.getVariables()
var_iterator = vars.createIterator()
var = var_iterator.Next()
while var:
    if var.GetName() == 'n_bkg':
        n_bkg = var.getVal()
    elif var.GetName() == 'n_sig':
        n_sig = var.getVal()
    var = var_iterator.Next()

l = ROOT.TLegend( 0.6, 0.65, 0.98, 0.98)
l.SetTextSize(0.04)
sigobj = plot.findObject("signal")
bkgobj = plot.findObject("background")
l.AddEntry( sigobj , "{0:.0f} true signal".format(sigsum), "l" )
l.AddEntry( sigobj, "#rightarrow {0:.0f} fitted".format(n_sig),"l")
l.AddEntry( bkgobj , "{0:.0f} true background".format(bkgsum), "l"  )
l.AddEntry( bkgobj , "#rightarrow {0:.0f} fitted".format(n_bkg), "l"  )

plot.Draw()
l.Draw()
c.Draw()

In [127]:
c.SaveAs("output/LikelihoodFit.eps")

Info in <TCanvas::Print>: eps file output/LikelihoodFit.eps has been created


# VBF Region

In [128]:
all = list(csv.reader(open(inputfile,"rb"), delimiter=','))
VBF = [variables for variables in all[1:] if (float(variables[6]) > 200 and float(variables[5]) > 2) ]

In [129]:
sig = [float(row[1]) for row in VBF[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in VBF[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in VBF[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in VBF[1:] if row[-1] == 'b' ]

sigsum = sum(sigweights)
bkgsum = sum(bkgweights)
print 'n signal {0}, n background {1}'.format(sigsum,bkgsum)

h1 = ROOT.TH1D("h1","signal histogram",20,0,200)
for i in range(len(sig)): h1.Fill(sig[i],sigweights[i])
h2 = ROOT.TH1D("h2","background histogram",20,0,200)
for i in range(len(bkg)): h2.Fill(bkg[i],bkgweights[i])
x = ROOT.RooRealVar("x","x",0,200)
l = ROOT.RooArgList(x)
signalhist = ROOT.RooDataHist("sighist", "sighist", l, h1)
sigpdf = ROOT.RooHistPdf("sigpdf","sigpdf",ROOT.RooArgSet(x),signalhist,0)
bkghist = ROOT.RooDataHist("bkghist", "bkghist", l, h2)
bkgpdf = ROOT.RooHistPdf("bkgpdf","bkgpdf",ROOT.RooArgSet(x),bkghist,0)
w = ROOT.RooWorkspace("w")
getattr(w,'import')(sigpdf)
getattr(w,'import')(bkgpdf)
w.var("x")
w.factory("SUM:model(n_sig[0.1*n_sig,10*n_sig]*sigpdf, n_bkg[.1*n_bkg,10*n_bkg]*bkgpdf)")
w.var("n_sig").setRange(0.1*sigsum,10*sigsum)
w.var("n_sig").setVal(sigsum)
w.var("n_bkg").setRange(.1*bkgsum,10*bkgsum)
w.var("n_bkg").setVal(bkgsum)
w.factory("ExtendPdf:background(bkgpdf,n_bkg)")
pdf = w.pdf("background")
x = w.var("x")
x.setBins(20)
data = pdf.generate(ROOT.RooArgSet(x))
data.SetName("data")
getattr(w,'import')(data)
data.Print()

n signal 525.136909244, n background 7794.33337887
RooDataSet::data[x] = 7794 entries


Warning in <TROOT::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).


In [130]:
mean = ROOT.RooRealVar("mean","Mean of Gaussian",0,200)
sigma = ROOT.RooRealVar("sigma","Width of Gaussian",40,0,200)
ztautau = ROOT.RooBreitWigner("ztautau","Breit-Wigner(x,mean,sigma)",x,mean,sigma)
coef0 = ROOT.RooRealVar("c0","coefficient #0",1.0,-1.,1)
coef1 = ROOT.RooRealVar("c1","coefficient #1",0.1,-1.,1)
coef2 = ROOT.RooRealVar("c2","coefficient #2",-0.1,-1.,1)
fake = ROOT.RooChebychev("fake","background p.d.f.",x,ROOT.RooArgList(coef0,coef1,coef2))
fbkg = ROOT.RooRealVar("fbkg","background fraction",0.9,0.,1.)
model = ROOT.RooAddPdf('prod','prod',ROOT.RooArgList(ztautau,fake),ROOT.RooArgList(fbkg))
mean.setVal(91.1876)
mean.setConstant(1)
model.fitTo(data,ROOT.RooFit.Range(0,200),ROOT.RooFit.PrintLevel(-1))

<ROOT.RooFitResult object at 0x(nil)>

In [131]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("Reconstructed Mass"))
plot.SetTitle("")
plot.GetYaxis().SetTitleOffset(1.)
plot.GetYaxis().SetTitleSize(0.05)
plot.GetXaxis().SetTitleSize(0.05)
plot.SetXTitle("MMC mass (GeV)")
data.plotOn(plot,ROOT.RooFit.Name("data"))
model.plotOn(plot,ROOT.RooFit.Name("ztautau"))
bkg_component = ROOT.RooArgSet(fake)
model.plotOn(plot,ROOT.RooFit.Components(bkg_component),ROOT.RooFit.LineStyle(2),ROOT.RooFit.Name("fake"))

l = ROOT.TLegend( 0.6, 0.6, 0.9, 0.9)
dataobj = plot.findObject("data")
zobj = plot.findObject("ztautau")
fakeobj = plot.findObject("fake")
l.AddEntry( dataobj , "Data", "pl" )
l.AddEntry( zobj , "Z#rightarrow#tau#tau", "l"  )
l.AddEntry( fakeobj , "Multijet", "l"  )
l.SetTextSizePixels(400)
plot.Draw()
l.Draw()
c.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: data (Potential memory leak).


In [132]:
c.SaveAs("output/MultiBackgroundVBF.eps")

Info in <TCanvas::Print>: eps file output/MultiBackgroundVBF.eps has been created


In [133]:
data = model.generate(ROOT.RooArgSet(x))
data.SetName("data")
r = model.fitTo(data, ROOT.RooFit.Save(True), ROOT.RooFit.Minimizer("Minuit2","Migrad"))
r.Print()

Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = 0
MnSeedGenerator: Initial state:   - FCN =                0 Edm =            0 NCalls =     21
MnSeedGenerator: Negative G2 found - new state:   - FCN =                0 Edm =            0 NCalls =     21
VariableMetric: start iterating until Edm is < 0.001
VariableMetric: Initial state   - FCN =                0 Edm =            0 NCalls =     21
VariableMetric: Iteration #   0 - FCN =                0 Edm =            0 NCalls =     21
VariableMetric: After Hessian   - FCN =                0 Edm =            0 NCalls =     30
VariableMetric: Iteration #   1 - FCN =                0 Edm =            0 NCalls =     30
Minuit2Minimizer : Invalid Minimum - status = 2
FVAL  = 0
Edm   = 0
Nfcn  = 30

  RooFitResult: minimized FCN value: 0, estimated distance to minimum: 0
                covariance matrix quality: Not calculated at all
                Status : MI

Info in <Minuit2>: MnHesse: 2nd derivative zero for Parameter  : name = c0
Info in <Minuit2>: MnHesse fails and will return diagonal matrix 
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Hesse is not valid
Info in <Minuit2>: MnHesse: 2nd derivative zero for Parameter  : name = c0
Info in <Minuit2>: MnHesse fails and will return diagonal matrix 
Info in <Minuit2>: Minuit2Minimizer::Hesse : Hesse failed - matrix is not valid
Info in <Minuit2>: MInuit2Minimizer::Hesse : hstatus = 3
Warning in <ROOT::Math::Fitter::CalculateHessErrors>: Error when calculating Hessian


In [134]:
xVBF = [variables for variables in all[1:] if (float(variables[6]) > 200 and float(variables[5]) > 2)]
sig = [float(row[1]) for row in xVBF[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in xVBF[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in xVBF[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in xVBF[1:] if row[-1] == 'b' ]

sumhist = ROOT.TH1D("sumhist","sum histogram",20,0,200)
h2 = ROOT.TH1D("h2","background histogram",20,0,200)
for i in range(len(sig)): 
    sumhist.Fill(sig[i],sigweights[i])
for i in range(len(bkg)): 
    sumhist.Fill(bkg[i],bkgweights[i])
    h2.Fill(bkg[i],bkgweights[i])
sumhist.GetSumw2()
h3 = ROOT.TH1D("h3","background systematic up",20,0,200)
h4 = ROOT.TH1D("h4","background systematic down",20,0,200)
for i in range(len(bkg)): h3.Fill(bkg[i])
for i in range(h2.GetNbinsX()):
    scale =(abs(h2.GetBinCenter(i+1)-91))/91
    thisbin = h2.GetBinContent(i+1)
    print 'scaling by ',scale
    h3.SetBinContent(i+1,thisbin*(1+scale))
    h4.SetBinContent(i+1,thisbin*(1-scale))

scaling by  0.945054945055
scaling by  0.835164835165
scaling by  0.725274725275
scaling by  0.615384615385
scaling by  0.505494505495
scaling by  0.395604395604
scaling by  0.285714285714
scaling by  0.175824175824
scaling by  0.0659340659341
scaling by  0.043956043956
scaling by  0.153846153846
scaling by  0.263736263736
scaling by  0.373626373626
scaling by  0.483516483516
scaling by  0.593406593407
scaling by  0.703296703297
scaling by  0.813186813187
scaling by  0.923076923077
scaling by  1.03296703297
scaling by  1.14285714286


Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h4 (Potential memory leak).


In [136]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("VBF Region"))
plot.SetTitle("")
plot.GetXaxis().SetTitleSize(0.05)
plot.GetYaxis().SetTitleSize(0.05)
plot.SetXTitle("MMC mass (GeV)")
pdf = w.pdf("model")
data = pdf.generate(ROOT.RooArgSet(x))
data.plotOn(plot)
pdf.plotOn(plot)
pdf.plotOn(plot,ROOT.RooFit.Components("bkgpdf"),ROOT.RooFit.Name("background"), ROOT.RooFit.LineStyle(2) )
pdf.plotOn(plot, ROOT.RooFit.Components("sigpdf"),ROOT.RooFit.Name("signal"), ROOT.RooFit.LineColor(2), ROOT.RooFit.LineStyle(2) )

vars = model.getVariables()
var_iterator = vars.createIterator()
var = var_iterator.Next()
while var:
    if var.GetName() == 'n_bkg':
        n_bkg = var.getVal()
    elif var.GetName() == 'n_sig':
        n_sig = var.getVal()
    var = var_iterator.Next()

l = ROOT.TLegend( 0.7, 0.5, .99, .99)
l.SetTextSize(0.03)
sigobj = plot.findObject("signal")
bkgobj = plot.findObject("background")
l.AddEntry( sigobj , "{0:.0f}({1:.0f}) signal".format(sigsum,n_sig), "l" )
l.AddEntry( bkgobj , "{0:.0f}({1:.0f}) background".format(bkgsum,n_bkg), "l"  )
l.AddEntry( dataobj , "Data", "pl" )
plot.Draw()
shapehist=sumhist.Clone()
for bin in range(shapehist.GetNbinsX()+1):
    h3bin = h3.GetBinContent(bin+1)
    h4bin = h4.GetBinContent(bin+1)
    shapehist.SetBinError(bin+1,abs(h3bin-h4bin)/2)
shapehist.SetFillColor(5)
shapehist.Scale((bkgsum+sigsum)/sumhist.Integral())
shapehist.SetStats(0)
shapehist.SetTitle("")
shapehist.Draw("same E4")
normhist=sumhist.Clone()
norm = (sumhist.Integral()-h2.Integral())/(0.003*h2.Integral())
print norm
for bin in range(normhist.GetNbinsX()+1):
#    normhist.SetBinContent(bin,normhist.GetBinContent(bin)+10)
    normhist.SetBinError(bin,norm)
normhist.SetFillColor(2)
normhist.Scale((bkgsum+sigsum)/sumhist.Integral())
normhist.SetFillStyle(3002)
normhist.Draw("same E4")
l.AddEntry( shapehist , "Shape Uncertainty", "F"  )
l.AddEntry( normhist , "Normalisation Uncertainty", "F"  )

plot.Draw("same")
#l1.Draw()
l.Draw()
c.Draw()

32.2736038785


In [137]:
c.SaveAs("output/VBFSystematics.pdf")

Info in <TCanvas::Print>: pdf file output/VBFSystematics.pdf has been created


In [123]:
xSR = [variables for variables in VBF[1:] if (float(variables[1]) > 90 and float(variables[1]) < 160) ]
sig = [float(row[1]) for row in xSR[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in xSR[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in xSR[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in xSR[1:] if row[-1] == 'b' ]

sumhist = ROOT.TH1D("sumhist","sum histogram",7,90,160)
h2 = ROOT.TH1D("h2","background histogram",7,90,160)
for i in range(len(sig)): 
    sumhist.Fill(sig[i],sigweights[i])
for i in range(len(bkg)): 
    sumhist.Fill(bkg[i],bkgweights[i])
    h2.Fill(bkg[i],bkgweights[i])
sumhist.GetSumw2()
h3 = ROOT.TH1D("h3","background systematic up",7,90,160)
h4 = ROOT.TH1D("h4","background systematic down",7,90,160)
for i in range(h2.GetNbinsX()):
    scale =(abs(h2.GetBinCenter(i+1)-91))/91
    thisbin = h2.GetBinContent(i+1)
    print 'scaling by ',scale
    h3.SetBinContent(i+1,thisbin*(1+scale))
    h4.SetBinContent(i+1,thisbin*(1-scale))

scaling by  0.043956043956
scaling by  0.153846153846
scaling by  0.263736263736
scaling by  0.373626373626
scaling by  0.483516483516
scaling by  0.593406593407
scaling by  0.703296703297


Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h4 (Potential memory leak).


In [101]:
SR = [variables for variables in VBF[1:] if (float(variables[1]) > 100 and float(variables[1]) < 150) ]

In [102]:
sig = [float(row[1]) for row in SR[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in SR[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in SR[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in SR[1:] if row[-1] == 'b' ]

sigsum = sum(sigweights)
bkgsum = sum(bkgweights)
print 'n signal {0}, n background {1}'.format(sigsum,bkgsum)

h1 = ROOT.TH1D("h1","signal histogram",5,100,150)
for i in range(len(sig)):     h1.Fill(sig[i],sigweights[i])

h2 = ROOT.TH1D("h2","background histogram",5,100,150)
for i in range(len(bkg)):     h2.Fill(bkg[i],bkgweights[i])
x = ROOT.RooRealVar("x","x",100,150)
l = ROOT.RooArgList(x)
signalhist = ROOT.RooDataHist("sighist", "sighist", l, h1)
sigpdf = ROOT.RooHistPdf("sigpdf","sigpdf",ROOT.RooArgSet(x),signalhist,0)
bkghist = ROOT.RooDataHist("bkghist", "bkghist", l, h2)
bkgpdf = ROOT.RooHistPdf("bkgpdf","bkgpdf",ROOT.RooArgSet(x),bkghist,0)
w = ROOT.RooWorkspace("w")
getattr(w,'import')(sigpdf)
getattr(w,'import')(bkgpdf)
w.var("x")
w.factory("SUM:model(n_sig[0.1*n_sig,10*n_sig]*sigpdf, n_bkg[.1*n_bkg,10*n_bkg]*bkgpdf)")
w.var("n_sig").setRange(0.1*sigsum,10*sigsum)
w.var("n_sig").setVal(sigsum)
n_bkg=w.var("n_bkg")
w.var("n_bkg").setRange(.1*bkgsum,10*bkgsum)
w.var("n_bkg").setVal(bkgsum)
w.factory("ExtendPdf:background(bkgpdf,n_bkg)")
w.factory("ExtendPdf:signal(sigpdf,n_sig)")
pdf = w.pdf("model")
x = w.var("x")
x.setBins(5)
data = pdf.generate(ROOT.RooArgSet(x))
data.SetName("data")
getattr(w,'import')(data)
data.Print()

n signal 421.876963784, n background 1819.96369349
RooDataSet::data[x,weight:weight] = 5 entries (2242 weighted)


Warning in <TROOT::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).


In [103]:
r = pdf.fitTo(data, ROOT.RooFit.Save(True), ROOT.RooFit.Minimizer("Minuit2","Migrad"))
r.Print()

Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = -6377.701219215
MnSeedGenerator: Initial state:   - FCN =  -6377.701219215 Edm =    0.0503311 NCalls =      9
VariableMetric: start iterating until Edm is < 0.001
VariableMetric: Initial state   - FCN =  -6377.701219215 Edm =    0.0503311 NCalls =      9
VariableMetric: Iteration #   0 - FCN =  -6377.701219215 Edm =    0.0503311 NCalls =      9
VariableMetric: Iteration #   1 - FCN =  -6377.784253647 Edm =     0.107839 NCalls =     15
VariableMetric: Iteration #   2 - FCN =  -6378.407803759 Edm =    0.0176669 NCalls =     23
VariableMetric: Iteration #   3 - FCN =  -6378.420713009 Edm =  0.000730936 NCalls =     29
VariableMetric: Iteration #   4 - FCN =  -6378.421523925 Edm =  3.73209e-08 NCalls =     35
VariableMetric: After Hessian   - FCN =  -6378.421523925 Edm =  3.24633e-08 NCalls =     47
VariableMetric: Iteration #   5 - FCN =  -6378.421523925 Edm =  3

Info in <Minuit2>: Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


In [104]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("Fit Signal + Background"))
pdf = w.pdf("model")
data = pdf.generate(ROOT.RooArgSet(x))
data.plotOn(plot)
pdf.plotOn(plot)
pdf.plotOn(plot,ROOT.RooFit.Components("bkgpdf"),ROOT.RooFit.Name("background"), ROOT.RooFit.LineStyle(2) )
pdf.plotOn(plot, ROOT.RooFit.Components("sigpdf"),ROOT.RooFit.Name("signal"), ROOT.RooFit.LineColor(2), ROOT.RooFit.LineStyle(2) )

pdf.paramOn(plot,ROOT.RooFit.Layout(0.6,0.9,0.85))

l = ROOT.TLegend( 0.6, 0.5, 0.899, 0.727775)
sigobj = plot.findObject("signal")
bkgobj = plot.findObject("background")
l.AddEntry( sigobj , "{0:.2f} signal events".format(sigsum), "l" )
l.AddEntry( bkgobj , "{0:.2f} background events".format(bkgsum), "l"  )

plot.Draw()
l.Draw()
c.Draw()

In [105]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("VBF Signal Region"))
plot.SetTitle("")
plot.GetYaxis().SetTitleOffset(1.)
plot.GetYaxis().SetTitleSize(0.05)
plot.GetXaxis().SetTitleSize(0.05)
plot.SetXTitle("MMC Mass (GeV)")
pdf = w.pdf("model")
data = pdf.generate(ROOT.RooArgSet(x))
data.plotOn(plot)
pdf.plotOn(plot)
pdf.plotOn(plot,ROOT.RooFit.Components("bkgpdf"),ROOT.RooFit.Name("background"), ROOT.RooFit.LineStyle(2) )
pdf.plotOn(plot, ROOT.RooFit.Components("sigpdf"),ROOT.RooFit.Name("signal"), ROOT.RooFit.LineColor(2), ROOT.RooFit.LineStyle(2) )

vars = pdf.getVariables()
var_iterator = vars.createIterator()
var = var_iterator.Next()
while var:
    if var.GetName() == 'n_bkg':
        n_bkg = var.getVal()
    elif var.GetName() == 'n_sig':
        n_sig = var.getVal()
    var = var_iterator.Next()

l = ROOT.TLegend( 0.7, 0.5, 0.99, 0.99)
l.SetTextSize(0.03)
sigobj = plot.findObject("signal")
bkgobj = plot.findObject("background")
l.AddEntry( sigobj , "{0:.0f}({1:.0f}) signal".format(sigsum,n_sig), "l" )
l.AddEntry( bkgobj , "{0:.0f}({1:.0f}) background".format(bkgsum,n_bkg), "l"  )
l.AddEntry( dataobj , "Data", "pl" )
plot.Draw()
shapehist=sumhist.Clone()
for bin in range(shapehist.GetNbinsX()+1):
    h3bin = h3.GetBinContent(bin+1)
    h4bin = h4.GetBinContent(bin+1)
    shapehist.SetBinError(bin+1,abs(h3bin-h4bin)/2)
shapehist.SetFillColor(5)
#shapehist.Scale((bkgsum+sigsum)/sumhist.Integral())
shapehist.Draw("same E2")
normhist=sumhist.Clone()
import math
norm = math.sqrt(sumhist.Integral()-h2.Integral())
print norm
for bin in range(normhist.GetNbinsX()+1):
#    normhist.SetBinContent(bin,normhist.GetBinContent(bin)+10)
    normhist.SetBinError(bin,norm)
normhist.SetFillColor(2)
#normhist.Scale((bkgsum+sigsum)/sumhist.Integral())
normhist.SetFillStyle(3002)
normhist.Draw("same E2")
l.AddEntry( shapehist , "Shape Uncertainty", "F"  )
l.AddEntry( normhist , "Normalisation Uncertainty", "F"  )

plot.Draw("same")
#l1.Draw()
l.Draw()
c.Draw()

37.3615790173


In [107]:
c.SaveAs("output/VBFSR.pdf")

Info in <TCanvas::Print>: pdf file output/VBFSR.pdf has been created


In [93]:
xCR = [variables for variables in VBF[1:] if (float(variables[1]) < 100) and (float(variables[1]) > 80)]
sig = [float(row[1]) for row in xCR[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in xCR[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in xCR[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in xCR[1:] if row[-1] == 'b' ]

sumhist = ROOT.TH1D("sumhist","sum histogram",4,80,100)
h2 = ROOT.TH1D("h2","background histogram",4,80,100)
for i in range(len(sig)): 
    sumhist.Fill(sig[i],sigweights[i])
for i in range(len(bkg)): 
    sumhist.Fill(bkg[i],bkgweights[i])
    h2.Fill(bkg[i],bkgweights[i])
sumhist.GetSumw2()
h3 = ROOT.TH1D("h3","background systematic up",4,80,100)
h4 = ROOT.TH1D("h4","background systematic down",4,80,100)
for i in range(h2.GetNbinsX()):
    scale =(abs(h2.GetBinCenter(i+1)-91))/91
    thisbin = h2.GetBinContent(i+1)
    print 'scaling by ',scale
    h3.SetBinContent(i+1,thisbin*(1+scale))
    h4.SetBinContent(i+1,thisbin*(1-scale))

scaling by  0.0934065934066
scaling by  0.0384615384615
scaling by  0.0164835164835
scaling by  0.0714285714286


Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sumhist (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h4 (Potential memory leak).


In [94]:
CR = [variables for variables in VBF[1:] if (float(variables[1]) < 100) and (float(variables[1]) > 80)  ]

In [95]:
sig = [float(row[1]) for row in CR[1:] if row[-1] == 's' ]
sigweights = [float(row[-2])*10 for row in CR[1:] if row[-1] == 's' ]
bkg = [float(row[1]) for row in CR[1:] if row[-1] == 'b' ]
bkgweights = [float(row[-2]) for row in CR[1:] if row[-1] == 'b' ]

sigsum = sum(sigweights)
bkgsum = sum(bkgweights)
print 'n signal {0}, n background {1}'.format(sigsum,bkgsum)

h1 = ROOT.TH1D("h1","signal histogram",4,80,100)
for i in range(len(sig)):     h1.Fill(sig[i],sigweights[i])

h2 = ROOT.TH1D("h2","background histogram",4,80,100)
for i in range(len(bkg)):     h2.Fill(bkg[i],bkgweights[i])
x = ROOT.RooRealVar("x","x",80,100)
l = ROOT.RooArgList(x)
signalhist = ROOT.RooDataHist("sighist", "sighist", l, h1)
sigpdf = ROOT.RooHistPdf("sigpdf","sigpdf",ROOT.RooArgSet(x),signalhist,0)
bkghist = ROOT.RooDataHist("bkghist", "bkghist", l, h2)
bkgpdf = ROOT.RooHistPdf("bkgpdf","bkgpdf",ROOT.RooArgSet(x),bkghist,0)
w = ROOT.RooWorkspace("w")
getattr(w,'import')(sigpdf)
getattr(w,'import')(bkgpdf)
w.var("x")
w.factory("SUM:model(n_sig[0.1*n_sig,10*n_sig]*sigpdf, n_bkg[.1*n_bkg,10*n_bkg]*bkgpdf)")
n_sig=w.var("n_sig")
w.var("n_sig").setRange(0.1*sigsum,10*sigsum)
w.var("n_sig").setVal(sigsum)
n_bkg=w.var("n_bkg")
w.var("n_bkg").setRange(.1*bkgsum,10*bkgsum)
w.var("n_bkg").setVal(bkgsum)
w.factory("ExtendPdf:background(bkgpdf,n_bkg)")
w.factory("ExtendPdf:background(sigpdf,n_sig)")
pdf = w.pdf("model")
x = w.var("x")
x.setBins(4)
data = pdf.generate(ROOT.RooArgSet(x))
data.SetName("data")
getattr(w,'import')(data)
data.Print()

n signal 49.7659558473, n background 1348.93556723
RooDataSet::data[x,weight:weight] = 4 entries (1399 weighted)


Warning in <TROOT::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h2 (Potential memory leak).


In [96]:
r = pdf.fitTo(data, ROOT.RooFit.Save(True), ROOT.RooFit.Minimizer("Minuit2","Migrad"))
r.Print()

Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
MnSeedGenerator: for initial parameters FCN = -4544.932535963
MnSeedGenerator: Initial state:   - FCN =  -4544.932535963 Edm =     0.032114 NCalls =      9
VariableMetric: start iterating until Edm is < 0.001
VariableMetric: Initial state   - FCN =  -4544.932535963 Edm =     0.032114 NCalls =      9
VariableMetric: Iteration #   0 - FCN =  -4544.932535963 Edm =     0.032114 NCalls =      9
VariableMetric: Iteration #   1 - FCN =  -4544.962662251 Edm =    0.0250623 NCalls =     15
VariableMetric: Iteration #   2 - FCN =  -4545.060780997 Edm =   0.00595872 NCalls =     22
VariableMetric: Iteration #   3 - FCN =   -4545.06450632 Edm =  4.09644e-05 NCalls =     28
VariableMetric: After Hessian   - FCN =   -4545.06450632 Edm =   4.5406e-05 NCalls =     40
VariableMetric: Iteration #   4 - FCN =   -4545.06450632 Edm =   4.5406e-05 NCalls =     40
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -4545.06450

Info in <Minuit2>: Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


In [97]:
c = ROOT.TCanvas()
plot = x.frame(ROOT.RooFit.Title("VBF Control Region"))
plot.SetTitle("")
plot.GetYaxis().SetTitleOffset(1.)
plot.GetYaxis().SetTitleSize(0.05)
plot.GetXaxis().SetTitleSize(0.05)
plot.SetXTitle("MMC Mass (GeV)")
pdf = w.pdf("model")
data = pdf.generate(ROOT.RooArgSet(x))
data.plotOn(plot)
pdf.plotOn(plot)
pdf.plotOn(plot,ROOT.RooFit.Components("bkgpdf"),ROOT.RooFit.Name("background"), ROOT.RooFit.LineStyle(2) )
pdf.plotOn(plot, ROOT.RooFit.Components("sigpdf"),ROOT.RooFit.Name("signal"), ROOT.RooFit.LineColor(2), ROOT.RooFit.LineStyle(2) )
vars = pdf.getVariables()
var_iterator = vars.createIterator()
var = var_iterator.Next()
while var:
    if var.GetName() == 'n_bkg':
        n_bkg = var.getVal()
    elif var.GetName() == 'n_sig':
        n_sig = var.getVal()
    var = var_iterator.Next()

l = ROOT.TLegend( 0.32, 0.15, 0.65, 0.65)
l.SetTextSize(0.03)
sigobj = plot.findObject("signal")
bkgobj = plot.findObject("background")
l.AddEntry( sigobj , "{0:.0f}({1:.0f}) signal".format(sigsum,n_sig), "l" )
l.AddEntry( bkgobj , "{0:.0f}({1:.0f}) background".format(bkgsum,n_bkg), "l"  )
l.AddEntry( dataobj , "Data", "pl" )
plot.Draw()
l.Draw()
shapehist=sumhist.Clone()
shapehist.SetFillColor(5)
shapehist.Scale((bkgsum+sigsum)/sumhist.Integral())
for bin in range(1,shapehist.GetNbinsX()+1):
    h3bin = h3.GetBinContent(bin)
    h4bin = h4.GetBinContent(bin)
    print 'setting error to ',abs(h3bin-h4bin)/2
    shapehist.SetBinError(bin,abs(h3bin-h4bin)/2)
shapehist.Draw("same E2")
normhist=sumhist.Clone()
norm = (sumhist.Integral()-h2.Integral())
print normhist.GetNbinsX()*norm
for bin in range(normhist.GetNbinsX()+1):
#    normhist.SetBinContent(bin,normhist.GetBinContent(bin)+10)
    normhist.SetBinError(bin,norm)
normhist.SetFillColor(2)
normhist.Scale((bkgsum+sigsum)/sumhist.Integral())
normhist.SetFillStyle(3002)
normhist.Draw("same E2")
l.AddEntry( shapehist , "Shape Uncertainty", "F"  )
l.AddEntry( normhist , "Normalisation Uncertainty", "F"  )
plot.Draw("same")
c.Draw()

setting error to  28.1569353029
setting error to  13.5154115069
setting error to  5.95386137286
setting error to  23.9206505707
199.063823389


In [99]:
c.SaveAs("output/VBFCR.pdf")

Info in <TCanvas::Print>: pdf file output/VBFCR.pdf has been created
